# No modificar, todo funciona

In [37]:
# Librerias necesarias
import pandas as pd
import os
import json

In [38]:
# Cargar los datos
df = pd.read_excel("../2. data/0 TimeLog (original, no modificar).xlsx")

In [39]:
# Copio para no modificar el original
tl = df.copy()

In [40]:
# Agrego LOS a cada fila y reordeno
tl["LOS"] = (tl["TF"] - tl["TI"])
tl = tl[["ID", "MS_GRD", "UBICACIÓN", "TI", "TF", "LOS", "HOSPITAL", "UNIDAD"]]

# Lista para guardar filas nuevas
new_rows = []

# Recorrer filas consecutivas
for i in range(len(tl) - 1):
    row_current = tl.iloc[i]
    row_next = tl.iloc[i + 1]

    # Verificar condiciones
    same_id = row_current['ID'] == row_next['ID']
    same_ms = row_current['MS_GRD'] == row_next['MS_GRD']
    time_gap = row_current['TF'] < row_next['TI']
    same_hospital = row_current['HOSPITAL'] == row_next['HOSPITAL']

    if same_id and same_ms and time_gap:
        # Crear fila intermedia
        new_row = {
            'ID': row_current['ID'],
            'MS_GRD': row_current['MS_GRD'],
            'UBICACIÓN': f"{row_current['UBICACIÓN']} -> {row_next['UBICACIÓN']}",
            'TI': row_current['TF'],
            'TF': row_next['TI'],
            'LOS': row_next['TI'] - row_current['TF'],
            'HOSPITAL': row_current['HOSPITAL'],
            'UNIDAD': row_current['UNIDAD']
        }
        new_rows.append((i + 1, new_row))  # guardar con índice de inserción

# Lista para guardar todas las filas nuevas (dicts)
new_rows_list = [fila[1] for fila in new_rows]

# Convertir todo a un DataFrame una sola vez
new_rows_df = pd.DataFrame(new_rows_list)

# Concatenar en una sola pasada
tl = pd.concat([tl, new_rows_df], ignore_index=True)

# Reordenar filas para meter filas entremedio
tl = tl.sort_values(["ID", "TI"])

# Resetear index pra quie se vea ordenado
tl = tl.reset_index(drop=True)

In [41]:
# Matriz de transición (las valide al azar con Disco y me dio lo mismo)
matrices = {}
resultados = {}

for hospital in range(1,4):
    resultados[hospital] = {}    
    for grd in range(1, 9):
        resultados[hospital][grd] = {}
        v3 = tl[(tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])) & 
            (tl["MS_GRD"] == grd) & 
            (tl["HOSPITAL"] == f"Hospital_{hospital}")].sort_values(["ID", "TI"])

        transiciones = {}

        for i in range(len(v3) - 1):
            row_current = v3.iloc[i]
            row_next = v3.iloc[i + 1]

            # Verificar condiciones
            same_id = row_current['ID'] == row_next['ID']
            actual = row_current['UNIDAD']
            siguiente = row_next['UNIDAD']
            distinta = actual != siguiente
            if same_id and distinta:
                if (actual, siguiente) in transiciones:
                    transiciones[(actual, siguiente)] += 1

                else:
                    transiciones[(actual, siguiente)] = 1

            # Solo para cuando esta en SDU_WARD al ser nodo de termino
            elif not same_id and actual == "SDU_WARD":
                if (actual, actual) in transiciones:
                    transiciones[(actual, actual)] += 1
                else:
                    transiciones[(actual, actual)] = 1



        unidades = ["OR","ICU", "SDU_WARD"]
        num = len(unidades) + 1
        matriz = []
        for i in range(num):
            matriz.append([0] * num)

        for i in range(1, num):
            matriz[0][i] = unidades[i - 1]
            matriz[i][0] = unidades[i - 1]

        for i in range(1, num):
            for j in range(1, num):
                if (matriz[i][0], matriz[0][j]) in transiciones:
                    matriz[i][j] = transiciones[(matriz[i][0], matriz[0][j])]
                else:
                    matriz[i][j] = 0

        for i in range(1, num):
            total = sum(matriz[i][1:])
            for j in range(1, num):
                    matriz[i][j] = matriz[i][j]/total
        
        matriz[0][0] = "Unidad"
        matriz[1][1] = 0.00000
        matriz[2][2] = 0.00000

        matrices[(hospital, grd)] = matriz

In [42]:
unidades = ["OR","ICU", "SDU_WARD"]
for hospital in range(1,4):
    for grd in range(1, 9):
        matriz = matrices[(hospital, grd)]
        for unidad in range(1, 4):
            resultados[hospital][grd][unidad] = {
                1: matriz[unidad][1],
                2: matriz[unidad][2],
                3: matriz[unidad][3]
            }

def save_dict_as_json(data_dict, filename, folder):
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist
    path = os.path.join(folder, filename)
    with open(path, 'w') as f:
        json.dump(data_dict, f, indent=4)
    print(f"Dictionary saved to: {path}")

# Save the results to a JSON file
save_dict_as_json(resultados, filename="matrices.json", folder="resultados incertidumbre")

Dictionary saved to: resultados incertidumbre/matrices.json


In [43]:
# Generar el texto para las matrices en LaTeX
for key, m in matrices.items():
    
    texto = f"""
\\begin{{table}}[H]
    \\centering
    \\begin{{tabular}}{{lcccc}}
        \\toprule
        {m[0][0]} & {m[0][1]} & {m[0][2]} & {m[0][3]} & TERMINO\\\\
        \\midrule
        {m[1][0]}       & 0.00000 & {round(m[1][2],5)} & {round(m[1][3],5)} & 0.00000\\\\
        {m[2][0]}       & {round(m[2][1],5)} & 0.00000 & {round(m[2][3],5)} & 0.00000\\\\
        {m[3][0]}       & {round(m[3][1],5)} & {round(m[3][2],5)} & 0.00000 & {round(m[3][3],5)}\\\\
        \\bottomrule
    \\end{{tabular}}
    \\caption{{Matriz de transición Hospital: {key[0]} y MS\\_GRD: {key[1]}}}
    \\label{{tab:matriz_transicion_Hospital{key[0]}_GRD{key[1]}}}
\\end{{table}}
    """

    texto = texto.replace("SDU_WARD", "SDU\\_WARD")
    # print(texto)